In [1]:
!pip install --upgrade qiskit
!pip install qiskit_optimization
!pip install qiskit_algorithms

In [2]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *

# qiskit-ibmq-provider has been deprecated.
# Please see the Migration Guides in https://ibm.biz/provider_migration_guide for more detail.
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options

# Loading your IBM Quantum account(s)
service = QiskitRuntimeService(channel="ibm_quantum")

# Invoke a primitive. For more details see https://qiskit.org/documentation/partners/qiskit_ibm_runtime/tutorials.html
# result = Sampler("ibmq_qasm_simulator").run(circuits).result()
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import random

from qiskit.algorithms import QAOA
from qiskit.algorithms.optimizers import COBYLA
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization import QuadraticProgram
from qiskit import Aer
from qiskit.utils import QuantumInstance


/tmp/ipykernel_983/817090657.py:21: DeprecationWarning: ``qiskit.algorithms`` has been migrated to an independent package: https://github.com/qiskit-community/qiskit-algorithms. The ``qiskit.algorithms`` import path is deprecated as of qiskit-terra 0.25.0 and will be removed no earlier than 3 months after the release date. Please run ``pip install qiskit_algorithms`` and use ``import qiskit_algorithms`` instead.
  from qiskit.algorithms import QAOA


In [3]:
def load_environment_data(grid_size=(3,3,3), waypoints=[(2,0,0), (0,2,2)], no_fly_zones=[((1,1,1), (1,1,1))]):
    """
    Load the environment data including the 3D grid, waypoints, and no-fly zones.
    
    Parameters:
    - grid_size: tuple(int, int, int), specifying the dimensions of the 3D grid (default: (10, 10, 10))
    - waypoints: list of tuple(int, int, int), specifying the waypoints the drone has to visit (default: [(2, 2, 2), (7, 7, 7), (2, 7, 2)])
    - no_fly_zones: list of tuple(start, end), specifying the no-fly zones, each represented by starting and ending coordinates (default: [((4, 4, 4), (5, 5, 5))])

    Returns:
    - grid: 3D NumPy array, representing the environment
    """

    # Initialize a 3D grid with dimensions as per grid_size
    grid = np.zeros(grid_size)

    # Mark the waypoints with '1'
    for x, y, z in waypoints:
        grid[x, y, z] = 1

    # Mark the no-fly zones with '-1'
    for zone in no_fly_zones:
        start, end = zone
        for x in range(start[0], end[0] + 1):
            for y in range(start[1], end[1] + 1):
                for z in range(start[2], end[2] + 1):
                    grid[x, y, z] = -1

    return grid


In [4]:
def initialize_drone_state(grid, start_position=(0, 0, 0)):
    """
    Initialize the drone's state at the starting position within the 3D grid.
    
    Parameters:
    - grid: 3D NumPy array, representing the environment
    - start_position: tuple(int, int, int), specifying the initial position of the drone (default: (0, 0, 0))

    Returns:
    - drone_position: tuple(int, int, int), representing the drone's current coordinates
    """
    
    # Ensure the starting position is not a waypoint or in a no-fly zone
    if grid[start_position] != 0:
        raise ValueError("Invalid starting position: Either a waypoint or a no-fly zone.")

    drone_position = start_position

    return drone_position


In [5]:
def calculate_cost(current_state, next_move, grid):
    """
    Load the 3D grid and calculate the cost for a move from current state to next state.

    Parameters:
    - current_state: tuple(int, int, int), specifying the current coordinates (x, y, z).
    - next_move: tuple(int, int, int), specifying the next coordinates (x, y, z).
    - grid: 3D NumPy array, representing the environment.

    Returns:
    - float: cost of the move, returns 'inf' if the move is not allowed.
    """
    if not constraints_check(next_move, grid):
        return float('inf')  # Assign infinite cost if move is not allowed
    
    # Using Euclidean distance as the cost metric
    cost = np.linalg.norm(np.array(next_move) - np.array(current_state))
    
    return cost


In [6]:
def constraints_check(move, grid):
    """
    Check constraints for a proposed move in the 3D grid.

    Parameters:
    - move: tuple(int, int, int), specifying the next coordinates (x, y, z).
    - grid: 3D NumPy array, representing the environment.

    Returns:
    - bool: True if move is allowed, otherwise False.

    """
    x, y, z = move
    grid_shape = grid.shape
    
    # Check if the move is inside the grid boundaries
    if x < 0 or x >= grid_shape[0] or y < 0 or y >= grid_shape[1] or z < 0 or z >= grid_shape[2]:
        return False  # Move is outside the grid
    
    # Check if the move enters a no-fly zone
    if grid[x, y, z] == -1:
        return False  # Move enters a no-fly zone
    
    return True  # Move is okay


In [7]:
def quantum_optimization(grid): #Uses Quantum Approximate Optimization Algorithm to find the path that minimizes the total cost
    """
    Find the optimal path through the 3D grid using Quantum Approximate Optimization Algorithm (QAOA).

    Parameters:
    - grid: 3D NumPy array, representing the environment.

    Returns:
    - list of tuple(int, int, int): list of coordinates forming the optimal path through the grid.

    """
    #Initialize Quadratic Program
    qp = QuadraticProgram()

    # Variable Declaration
    for i in range(grid.shape[0]):
        for j in range(grid.shape[1]):
            for k in range(grid.shape[2]):
                qp.binary_var(name=f"x_{i}_{j}_{k}")

    # Create Quadratic Program 
    linear = {}
    quadratic = {}
    for i in range(grid.shape[0]):
        for j in range(grid.shape[1]):
            for k in range(grid.shape[2]):
                current_state = (i, j, k)
                for dx in [-1, 0, 1]:
                    for dy in [-1, 0, 1]:
                        for dz in [-1, 0, 1]:
                            next_move = (i + dx, j + dy, k + dz)
                            if constraints_check(next_move, grid):  # Check if move is allowed
                                cost = calculate_cost(current_state, next_move, grid)
                                var1 = f"x_{i}_{j}_{k}"
                                var2 = f"x_{i+dx}_{j+dy}_{k+dz}"
                                if var1 != var2:
                                    quadratic[(var1, var2)] = cost
    qp.minimize(linear=linear, quadratic=quadratic)
    
    # Set up quantum backend
    backend = Aer.get_backend('qasm_simulator')
    quantum_instance = QuantumInstance(backend)
    
    # Initialize COBYLA and QAOA 
    optimizer = COBYLA()
    qaoa = QAOA(optimizer=optimizer, quantum_instance=quantum_instance)

    # Initialize MinimumEigenOptimizer
    meo = MinimumEigenOptimizer(qaoa)

    # Solve the QuadraticProgram
    result = meo.solve(qp)
    
    # Extract Optimal Path
    optimal_path = []
    for i in range(grid.shape[0]):
        for j in range(grid.shape[1]):
            for k in range(grid.shape[2]):
                if result.variables_dict.get(f"x_{i}_{j}_{k}") == 1:
                    optimal_path.append((i, j, k))

    return optimal_path

In [9]:
grid = load_environment_data()
optimal_path = quantum_optimization(grid)
print("Optimal Path:", optimal_path)

/tmp/ipykernel_983/855401782.py:42: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  quantum_instance = QuantumInstance(backend)
/tmp/ipykernel_983/855401782.py:46: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.qaoa.QAOA`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.QAOA``. See https://qisk.it/algo_migration for a migration guide.
  qaoa = QAOA(optimizer=optimizer, quantum_instance=quantum_instance)


Optimal Path: [(0, 1, 2), (1, 0, 0), (1, 2, 0), (2, 0, 1), (2, 2, 2)]
